# 3. Stationary Distribution of Markov Chain

**Loading data**
`load_data` function will help in loading the data from the `P_008.txt` file.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

def load_data(filename):
  P = []
  with open(filename, 'r') as file:
    row = []
    for line in file:
      row.append([str(value) for value in line.split(',')])
    for r in row:
      P.append([float(value) for value in r[0].split(' ')])
  return np.array(P)


### Direct Method

Given a transition matrix $P$ of size $100 \times 100$, the stationary distribution $\pi$ satisfies:

1. **Stationarity Condition**:
   $$
   \pi^T P = \pi^T \quad \Rightarrow \quad \pi^T (P - I) = 0
   $$
   This gives $100$ equations for $100$ unknowns.

2. **Normalization Condition**:
   $$
   \pi^T 1 = 1
   $$
   Ensures the sum of $\pi$ components equals 1.

#### Number of Equations and Variables
- $100$ equations from stationarity.
- $1$ equation from normalization.

Thus, there are $101$ equations and $100$ variables.

#### Solving the System
The system is overdetermined, but the normalization condition adjusts it. Solve:
$$
A \pi = b
$$
where $A = P^T - I$ and $b[-1] = 1$, yielding the unique stationary distribution $\pi$.


In [ ]:
def compute_stationary_direct(P):
    n = P.shape[0]
    b = np.zeros(n)
    A = P.T - np.eye(n) # A = (P - I)
    # replacing the last equation with the constraint because Sum of π should be 1
    A[-1, :] = 1
    b[-1] = 1
    return np.dot(np.linalg.inv(A), b)

### Power Method for Stationary Distribution
This `compute_stationary_power` iteratively computes the stationary distribution $\pi$ of a transition matrix $P$ using the Power Method as discussed in class (and in lecture notes), stopping when the change between iterations is smaller than a tolerance or the maximum iterations are reached.


In [ ]:
def compute_stationary_power(P, tol=1e-8, max_iter=1000):
    n = P.shape[0]
    pi = np.ones(n) / n
    for i in range(max_iter):
        new_pi = pi @ P
        if np.linalg.norm(new_pi - pi, 1) < tol:
            return new_pi, i
        pi = new_pi
    return pi, max_iter

#### Veryfying Distribution
The `verify_stationary_distribution` function computes the error between the computed stationary distribution $\pi$ and the actual stationary distribution using the L1 norm. It also returns the sum of the components of $\pi$ to ensure it equals 1.


In [ ]:
def verify_stationary_distribution(P, pi):
    error = np.linalg.norm(pi @ P - pi, 1)
    sum_pi = np.sum(pi)
    return error, sum_pi


### Calculation and Comparision Results

1. **Direct Method**: Computes the stationary distribution $\pi$ with its error and execution time.
2. **Power Method**: Computes $\pi$ iteratively, showing convergence iterations and execution time.
3. **Comparison**: Compares errors and execution times of the direct and power methods.
4. **Accuracy**: Displays the errors for both methods.
5. **Computational Efficiency**: Compares the execution times of the two methods.


In [ ]:

P = load_data('P_008.txt')

# Direct method
start_time = time.time()
pi_direct = compute_stationary_direct(P)
time_direct = time.time() - start_time
error_direct, sum_direct = verify_stationary_distribution(P, pi_direct)

# Power method
start_time = time.time()
pi_power, iterations_power = compute_stationary_power(P)
time_power = time.time() - start_time
error_power, sum_power = verify_stationary_distribution(P, pi_power)

print("Direct Method:")
print("Stationary Distribution (π):", pi_direct)
print("Error (||π⊤P - π⊤||₁):", error_direct)
print("Sum of π:", sum_direct)
print("Execution Time:", time_direct, "seconds")

print("\nPower Method:")
print("Stationary Distribution (π):", pi_power)
print("Error (||π⊤P - π⊤||₁):", error_power)
print("Sum of π:", sum_power)
print("Iterations for Convergence:", iterations_power)
print("Execution Time:", time_power, "seconds")

print("\nComparison:")
print("Accuracy:")
print("Direct method error:", error_direct)
print("Power method error:", error_power)


print("\nComputational Efficiency:")
print("Direct method time:", time_direct)
print("Power method time:", time_power)


Direct Method:
Stationary Distribution (π): [0.00881672 0.01038619 0.0093248  0.00986107 0.00890843 0.00905618
 0.0104268  0.01008536 0.01013856 0.01008627 0.00946903 0.01005004
 0.01039549 0.01107021 0.01005826 0.01018863 0.01011913 0.01053488
 0.00946717 0.00913167 0.01014592 0.01096579 0.00956576 0.00953922
 0.01043032 0.01004577 0.00949422 0.00966776 0.01074929 0.00994642
 0.00992029 0.00965172 0.00958922 0.01064684 0.00985292 0.0099738
 0.01104734 0.00955717 0.00868335 0.010698   0.00861039 0.01093095
 0.01110307 0.00933843 0.01006935 0.0110716  0.00990908 0.0094745
 0.01059182 0.00980599 0.00999269 0.01071129 0.01099232 0.01089428
 0.0098782  0.01039944 0.01050427 0.00930321 0.00943418 0.01094622
 0.00992245 0.00945618 0.01013478 0.0099235  0.01011309 0.00922796
 0.00913135 0.01025017 0.00965613 0.01092748 0.00947531 0.00938851
 0.00985814 0.01027037 0.00991634 0.00950987 0.0093776  0.00981229
 0.01068009 0.00926201 0.00911516 0.00973615 0.01122639 0.01096509
 0.01032183 0.010520

### Impact of Matrix Size (Power Method)

This section tests the effect of matrix size on the Power Method for computing the stationary distribution. Matrix sizes $( 20 \times 20 )$, $( 50 \times 50 )$, $( 70 \times 70 )$, and $( 100 \times 100 )$ are used. For each size:
- A submatrix is extracted and normalized.
- The stationary distribution is computed, and the time, iterations, and error are printed.


In [ ]:
matrix_sizes = [20,50,70,100]
for size in matrix_sizes:
  if size == 100:
    P_sub = P
  else:
    start_index = (100 - size) // 2
    P_sub = P[start_index:start_index+size, start_index:start_index+size]
    P_sub = P_sub / np.sum(P_sub, axis=1, keepdims = True)

  start_time = time.time()
  pi_power_sub, iterations_power_sub = compute_stationary_power(P_sub)
  time_power_sub = time.time() - start_time

  error_power_sub, sum_power_sub = verify_stationary_distribution(P_sub, pi_power_sub)

  print(f"\nMatrix Size: {size}x{size}")
  print("Iterations:", iterations_power_sub)
  print("Time:", time_power_sub)
  print("Error", error_power_sub)


Matrix Size: 20x20
Iterations: 8
Time: 0.00024127960205078125
Error 4.6355873306591633e-10

Matrix Size: 50x50
Iterations: 7
Time: 0.00021910667419433594
Error 2.1202204228409904e-10

Matrix Size: 70x70
Iterations: 6
Time: 0.00018215179443359375
Error 4.958074836108883e-10

Matrix Size: 100x100
Iterations: 6
Time: 0.0002276897430419922
Error 1.1222005789901512e-10
